In [1]:
import pandas as pd
import sys
import numpy as np
from scipy.misc import imshow, imresize
import imageio
import os
from PIL import Image

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Concatenate, Reshape
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, Input, Lambda
from keras.models import Model

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras import backend as K
K.tensorflow_backend._get_available_gpus()


ModuleNotFoundError: No module named 'keras'

In [10]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

In [2]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


In [3]:
data = np.load("/atlas/u/esheehan/wikipedia_project/dataset/GUF_dataset/data/AfricaArticleClustersUpdated.npy")
titleMap = pd.read_csv('/atlas/u/esheehan/wikipedia_project/CNN/Africa_Image_Coordinates.csv').dropna()
imageRoot = "/atlas/u/esheehan/wikipedia_project/dataset/image_dataset/Africa_Images/"

In [4]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [5]:
def load_data():
    # Title ex: 0,""Pedro Arrupe" Political Training Institute",institute,38.13750555555556,13.333297222222226
    # Image format location: imageRoot + category + id
    
    print("Num examples: ", len(data))
    noImage = 0 
    
    X,Y = [], []
    for i in data:
        if i[1] == "NGA":
            closest3All = i[6][:3]
            closest3Filename = []
            for d in closest3All:
                f = titleMap[titleMap['name'] == d[1]]
                if f.empty:
#                     print("No file found for: ", d[1], file=sys.stderr)
                    f = titleMap[titleMap['name'] == d[1] + "_0"]
                    if f.empty:
                        noImage += 1 
                        continue 
                categoryName = f["category"].iloc[0]
                categoryName.replace(" ", "\ ")
                curFilename = imageRoot + categoryName + "/" +  str(f["id"].iloc[0]) + ".jpeg"
#                 print(curFilename)
                closest3Filename.append(curFilename)   
                # Currently returns filenames, should load them as images??
            if len(closest3Filename) == 3:
                curImages = []
                for file in closest3Filename:
                    im = Image.open(file).convert('RGB')
                    im = im.resize((224,224))
                    im2arr = np.array(im) # im2arr.shape: height x width x channel
                    curImages.append(im2arr)
                X.append(curImages)
                Y.append(i[2])
            

    print("No images found for {} examples.".format(noImage))
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=1234, shuffle=True)
    X_train, X_test, Y_train, Y_test = np.asarray(X_train), np.asarray(X_test), np.asarray(Y_train), np.asarray(Y_test)

    print("X_train Shape: ", X_train.shape)
    print("X_test Shape: ", X_test.shape)
    print("Y_train Shape: ", Y_train.shape)
    print("Y_test Shape: ", Y_test.shape)

    print("Data Loaded") 
    return X_train, X_test, Y_train, Y_test

In [6]:
X_train, X_test, Y_train, Y_test = load_data()

# X_train1, X_train2, X_train3 = splitX(X_train)
# X_test1, X_test2, X_test3 = splitX(X_test)
print("Dataset Loaded")

Num examples:  44193
No images found for 2681 examples.
X_train Shape:  (1397, 3, 224, 224, 3)
X_test Shape:  (350, 3, 224, 224, 3)
Y_train Shape:  (1397,)
Y_test Shape:  (350,)
Data Loaded
Dataset Loaded


In [30]:
def load_model():
    
    np.random.seed(1234)
    
    bigInput = Input(shape=(3,224,224,3), dtype='float32', name='bigInput') 
    input1 = Lambda(lambda x: x[:,0])(bigInput)
    input2 = Lambda(lambda x: x[:,1])(bigInput)
    input3 = Lambda(lambda x: x[:,2])(bigInput)
    
    print(bigInput.shape, input1.shape, input2.shape, input3.shape)
    
    # INPUT 1
    x = Conv2D(7, kernel_size=(3, 3), activation='sigmoid', input_shape=(224,224,3))(input1)
    x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
    x = Conv2D(3, (3, 3), activation='sigmoid')(x)
    x = Flatten()(x)

    # INPUT 2
    a = Conv2D(7, kernel_size=(3, 3), activation='sigmoid', input_shape=(224,224,3))(input2)
    a = MaxPooling2D(pool_size=(2, 2))(a)
#     a = Dropout(0.25)(a)
    a = Conv2D(3, (3, 3), activation='sigmoid')(a)
    a = Flatten()(a)
    
    # INPUT 3
    b = Conv2D(7, kernel_size=(3, 3), activation='sigmoid', input_shape=(224,224,3))(input3)
    b = MaxPooling2D(pool_size=(2, 2))(b)
#     b = Dropout(0.25)(b)
    b = Conv2D(3, (3, 3), activation='sigmoid')(b)
    b = Flatten()(b)
    
    c = keras.layers.concatenate([x, a, b])
#     print(c.shape)
    c = Dropout(0.2)(c)
    c = Dense(64)(c)
    main_output = Dense(1, name='main_output')(c)
    print(main_output.shape)
    
#     model = Model(inputs=[input1, input2, input3], outputs=[main_output])
    model = Model(inputs=[bigInput], outputs=[main_output])
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[coeff_determination,'mae'])
    print(model.summary())
    return model



In [31]:

def run_model():

    reg = load_model()
    print("Model Loaded")
    reg= KerasRegressor(build_fn=load_model, epochs=50, verbose=1,validation_split=0.2)

    print("Testing model")
    reg.fit(X_train, Y_train)
    prediction=reg.predict(X_test)
    print("R2: ", r2_score(Y_test, prediction))
    print("Pearson's r: ", pearsonr(Y_test, prediction))
    result = np.sqrt(mean_squared_error(Y_test, prediction))
    print("Testing RMSE: {}".format(result))


In [32]:

if __name__ == "__main__":

    run_model()



(?, 3, 224, 224, 3) (?, 224, 224, 3) (?, 224, 224, 3) (?, 224, 224, 3)
(?, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bigInput (InputLayer)           (None, 3, 224, 224,  0                                            
__________________________________________________________________________________________________
lambda_34 (Lambda)              (None, 224, 224, 3)  0           bigInput[0][0]                   
__________________________________________________________________________________________________
lambda_35 (Lambda)              (None, 224, 224, 3)  0           bigInput[0][0]                   
__________________________________________________________________________________________________
lambda_36 (Lambda)              (None, 224, 224, 3)  0           bigInput[0][0]                   
_______________________________

Train on 1117 samples, validate on 280 samples
Epoch 1/50
1117/1117 [==============================] - 165s 148ms/step - loss: 52996.4621 - coeff_determination: -103150.5735 - mean_absolute_error: 170.3931 - val_loss: 6047.5024 - val_coeff_determination: -12274.9103 - val_mean_absolute_error: 77.7610
Epoch 2/50
1117/1117 [==============================] - 162s 145ms/step - loss: 917.3761 - coeff_determination: -1979.7070 - mean_absolute_error: 24.4061 - val_loss: 59.6635 - val_coeff_determination: -119.9507 - val_mean_absolute_error: 7.6893
Epoch 3/50
1117/1117 [==============================] - 162s 145ms/step - loss: 21.9600 - coeff_determination: -47.7539 - mean_absolute_error: 3.8137 - val_loss: 4.2332 - val_coeff_determination: -7.5642 - val_mean_absolute_error: 1.9232
Epoch 4/50
1117/1117 [==============================] - 162s 145ms/step - loss: 1.2696 - coeff_determination: -1.6138 - mean_absolute_error: 0.8951 - val_loss: 0.5504 - val_coeff_determination: -0.0577 - val_mean_ab

KeyboardInterrupt: 